# AOC2022

## Day 23 / Part 1 / Unstable Diffusion

Problem Description: https://adventofcode.com/2022/day/23

Input: [Example](aoc2022_day23_example.txt)

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 23, part 1."""
import logging
import sys
import numpy as np

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day23_example.txt"

In [3]:
def build_grove_map(elf_pos_set):
    """Build the grove map from a set of elf positions."""
    min_y_pos = min(elf_pos[0] for elf_pos in elf_pos_set)
    max_y_pos = max(elf_pos[0] for elf_pos in elf_pos_set)
    min_x_pos = min(elf_pos[1] for elf_pos in elf_pos_set)
    max_x_pos = max(elf_pos[1] for elf_pos in elf_pos_set)

    grove_map = np.zeros(
        (max_y_pos - min_y_pos + 1, max_x_pos - min_x_pos + 1)
    )

    for elf_pos in elf_pos_set:
        grove_map[
            elf_pos[0]-min_y_pos, elf_pos[1]-min_x_pos
        ] = 1
    return grove_map


def calc_next_pos(elf_pos_set, elf_pos, order):
    """
    Calculate the next position of an elf starting at elf_pos
    with a given direction order.
    """
    y_pos, x_pos = elf_pos
    candidates = []
    for order_id in range(order, order+4):
        if order_id % 4 == 0:
            # check north
            if (
                (y_pos-1, x_pos-1) not in elf_pos_set and
                (y_pos-1, x_pos) not in elf_pos_set and
                (y_pos-1, x_pos+1) not in elf_pos_set
            ):
                candidates.append((y_pos-1, x_pos))
        elif order_id % 4 == 1:
            # check south
            if (
                (y_pos+1, x_pos-1) not in elf_pos_set and
                (y_pos+1, x_pos) not in elf_pos_set and
                (y_pos+1, x_pos+1) not in elf_pos_set
            ):
                candidates.append((y_pos+1, x_pos))
        elif order_id % 4 == 2:
            # check west
            if (
                (y_pos-1, x_pos-1) not in elf_pos_set and
                (y_pos, x_pos-1) not in elf_pos_set and
                (y_pos+1, x_pos-1) not in elf_pos_set
            ):
                candidates.append((y_pos, x_pos-1))
        elif order_id % 4 == 3:
            # check east
            if (
                (y_pos-1, x_pos+1) not in elf_pos_set and
                (y_pos, x_pos+1) not in elf_pos_set and
                (y_pos+1, x_pos+1) not in elf_pos_set
            ):
                candidates.append((y_pos, x_pos+1))

    if len(candidates) in [0, 4]:
        return None
    return candidates[0]


def do_round(elf_pos_set, order):
    """Execute one round of the time-consuming process."""
    next_elf_pos_map = {}
    for elf_pos in elf_pos_set:
        next_elf_pos = calc_next_pos(elf_pos_set, elf_pos, order)
        if next_elf_pos not in next_elf_pos_map:
            next_elf_pos_map[next_elf_pos] = []
        next_elf_pos_map[next_elf_pos].append(elf_pos)
    for next_elf_pos, prev_elf_pos_list in next_elf_pos_map.items():
        if len(prev_elf_pos_list) == 1:
            prev_elf_pos = prev_elf_pos_list[0]
            elf_pos_set.remove(prev_elf_pos)
            elf_pos_set.add(next_elf_pos)

In [4]:
def main():
    """Main function to solve puzzle."""
    elf_pos_set = set()
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        for y_pos, line in enumerate(
            [line.rstrip() for line in file_obj.readlines()]
        ):
            for x_pos, cell in enumerate(line):
                if cell == "#":
                    elf_pos_set.add((y_pos, x_pos))

    LOGGER.debug("run process...")
    for rid in range(10):
        LOGGER.debug(
            "  grove map after round %s:\n  %s\n",
            rid+1, str(build_grove_map(elf_pos_set)).replace("\n", "\n  ")
        )
        do_round(elf_pos_set, rid % 4)

    LOGGER.debug(
        "  grove map after round %s:\n  %s\n",
        rid+1, str(build_grove_map(elf_pos_set)).replace("\n", "\n  ")
    )

    min_y_pos, min_x_pos = 9999, 9999
    max_y_pos, max_x_pos = 0, 0
    for elf_pos in elf_pos_set:
        min_y_pos = min(elf_pos[0], min_y_pos)
        min_x_pos = min(elf_pos[1], min_x_pos)
        max_y_pos = max(elf_pos[0], max_y_pos)
        max_x_pos = max(elf_pos[1], max_x_pos)

    print(
        "solution: " + str(
            (max_y_pos-min_y_pos+1) * (max_x_pos-min_x_pos+1) -
            len(elf_pos_set)
        )
    )

In [5]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 110
